# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at mlta-2022-spring@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1H5ZONrb2LMOCixLY7D5_5-7LkIaXO6AGEaV2mRdTOMY/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2.5hrs
  

## Download Dataset

In [2]:
# Download link 1
!gdown --id '1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1qwjbRjq481lHsnTrrF4OjKQnxzgoLEFR' --output hw7_data.zip

# Download Link 3 (if the above link fails) 
# !gdown --id '1QXuWjNRZH6DscSd6QcRER0cnxmpZvijn' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb
To: /content/hw7_data.zip
100% 9.57M/9.57M [00:00<00:00, 249MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          
Sun May  1 00:13:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [3]:
# You are allowed to change version     of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.1 MB 11.7 MB/s 
     |████████████████████████████████| 3.3 MB 50.3 MB/s 
     |████████████████████████████████| 895 kB 46.9 MB/s 


## Import Packages

In [4]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast, get_linear_schedule_with_warmup

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [5]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
# GPU 在不需要這麼高精度的運算時可以轉換成 float_16（加速）
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

     |████████████████████████████████| 47 kB 3.2 MB/s 


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Model and Tokenizer




 

In [8]:
model = BertForQuestionAnswering.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large").to(device)
tokenizer = BertTokenizerFast.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large")

# hfl/chinese-macbert-base > hfl/chinese-roberta-wwm-ext(0.76) > hfl/chinese-bert-wwm(0.741)
# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Downloading:   0%|          | 0.00/669 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [9]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [10]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False, return_offsets_mapping=True)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False, return_offsets_mapping=True)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False, return_offsets_mapping=True)
# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

dev_paragraphs = [i.replace(' ','✔').replace('\u200b','✦').replace('\u200e', '☺').replace('\u3000', '☆').replace('#','●') for i in dev_paragraphs]
test_paragraphs = [i.replace(' ','✔').replace('\u200b','✦').replace('\u200e', '☺').replace('\u3000', '☆').replace('#','●') for i in test_paragraphs]


## Dataset and Dataloader

In [11]:
import random

class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 300
        self.counter = 0
        
        ##### TODO: Change value of doc_stride #####
        # self.doc_stride = 150
        self.doc_stride = 64
        # 60 => 0.743
        # 75 => 0.742
        # 35 => 0.747

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]
        self.counter = self.counter + 1
        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn
        # input_id, token, attention_mask
        # 每次 random 一個數字當作 offset
        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])
            # A single window is obtained by slicing the portion of paragraph containing the answer
            offset = random.randint(0, 299-(answer_end_token - answer_start_token))
            # 在 0（起始）與 
            # mid 往左二分之一的最大 paragraph 長度（一般情況）、總長-最大 paragraph（最後）=> 兩者較小者
            # 上述兩者中取最大值
            if(self.counter % 2) == 0:
              paragraph_start = max(0, min(answer_start_token - (offset), len(tokenized_paragraph) - self.max_paragraph_len))
              paragraph_end = min(paragraph_start + self.max_paragraph_len, len(tokenized_paragraph))
            else:
              paragraph_end = min(len(tokenized_paragraph), max(answer_end_token + (offset), 0 + self.max_paragraph_len))
              paragraph_start = max(0, paragraph_end - self.max_paragraph_len)
            # print(paragraph_start, answer_start_token, answer_end_token, paragraph_end)

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask


train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [12]:
def evaluate(data, output, paragraph_index, doc_stride, is_test, n_best):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    # check 是否存在 start index 比 end index 更後面的問題

    # 

    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    start_index_in_paragraph = 0
    end_index_in_paragraph = 0
    best_of_n_best_prob = -np.inf

    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position

        mask = data[1][0][k].bool() &  data[2][0][k].bool() # token type & attention mask
        masked_output_start = torch.masked_select(output.start_logits[k], mask.to(device))[:-1] # -1 is [SEP]
        # print(f'start: {_get_best_indexes(masked_output_start, 5)}')
        n_best_start_index = _get_best_indexes(masked_output_start, n_best)
        # start_prob, start_index = torch.max((masked_output_start), dim=0)
        # print(start_index)
        masked_output_end = torch.masked_select(output.end_logits[k], mask.to(device))[:-1] # -1 is [SEP]
        # for i in _get_best_indexes(masked_output_start, 5):
        #   print(masked_output_start[i])
        n_best_end_index = _get_best_indexes(masked_output_end, n_best)
        # print(f'end: {_get_best_indexes(masked_output_end, 5)}')
        # end_prob, end_index = torch.max((masked_output_end), dim=0)
        # print(end_index)

        # prob = start_prob + end_prob
        masked_data = torch.masked_select(data[0][0][k], mask)[:-1]
        for i in n_best_start_index:
          for j in n_best_end_index:
            if i > j :
              continue
            elif ((masked_output_start[i] + masked_output_end[j]) > best_of_n_best_prob) & ((j - i)<=40):
              best_of_n_best_prob = masked_output_start[i] + masked_output_end[j]
              start_index_in_paragraph = i + doc_stride*k
              end_index_in_paragraph = j + doc_stride*k
              answer = tokenizer.decode(masked_data[i : j + 1])
              # print(i, j)

        
        # Replace answer if calculated probability is larger than previous windows
        # done!!
        # if (prob > max_prob) & (start_index < end_index) & ((end_index - start_index)<=40):
        #     max_prob = prob
        #     start_index_in_paragraph = start_index + doc_stride*k
        #     end_index_in_paragraph = end_index + doc_stride*k
        #     # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
        #     answer = tokenizer.decode(masked_data[start_index : end_index + 1])
        #     print(start_index, end_index)

    if ('[UNK]' in answer) & (is_test == False):
      # print(tokenizer.decode(data[0][0][global_k]).replace(' ', ''))
      # print(dev_paragraphs[paragraph_index])
      # print(f'original answer: {answer}')
      # print(start_index_in_paragraph)
      # print(end_index_in_paragraph)
      # print(dev_paragraphs_tokenized['offset_mapping'][paragraph_index][start_index_in_paragraph])
      # print(dev_paragraphs_tokenized['offset_mapping'][paragraph_index][end_index_in_paragraph])
      start = dev_paragraphs_tokenized['offset_mapping'][paragraph_index][start_index_in_paragraph][0]
      end = dev_paragraphs_tokenized['offset_mapping'][paragraph_index][end_index_in_paragraph][1]
      answer = dev_paragraphs[paragraph_index][start:end].replace('✔', ' ').replace('✦','\u200b').replace('☺','\u200e').replace('☆','\u3000').replace('●','#')
      # print(f'final answer: {dev_paragraphs[paragraph_index][start:end]}')
    elif ('[UNK]' in answer) & (is_test == True):
      # print(f'original answer: {answer}')
      # print(start_index_in_paragraph)
      # print(end_index_in_paragraph)
      # print(test_paragraphs_tokenized['offset_mapping'][paragraph_index][start_index_in_paragraph])
      # print(test_paragraphs_tokenized['offset_mapping'][paragraph_index][end_index_in_paragraph])
      start = test_paragraphs_tokenized['offset_mapping'][paragraph_index][start_index_in_paragraph][0]
      end = test_paragraphs_tokenized['offset_mapping'][paragraph_index][end_index_in_paragraph][1]
      answer = test_paragraphs[paragraph_index][start:end].replace('✔', ' ').replace('✦','\u200b').replace('☺','\u200e').replace('☆','\u3000').replace('●','#')
      # print(f'final answer: {test_paragraphs[paragraph_index][start:end]}')


    # print(f'final answer: {answer}')
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    if ('「' == answer[0]) & ('」' != answer[-1]):
      answer = answer + '」'
    if ('」' == answer[-1]) & ('「' != answer[0]):
      answer = '「' + answer
    if ('『' == answer[0]) & ('』' != answer[-1]):
      answer = answer + '』'
    if ('』' == answer[-1]) & ('『' != answer[0]):
      answer = '『' + answer
    
    return answer.replace(' ','')

    



In [13]:
def _get_best_indexes(logits, n_best_size):
    """Get the n-best logits from a list."""
    index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)

    best_indexes = []
    for i in range(len(index_and_score)):
        if i >= n_best_size:
            break
        best_indexes.append(index_and_score[i][0])
    return best_indexes


## Training

In [14]:
num_epoch = 3
validation =  False
logging_step = 200
learning_rate = 3e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
n_best = 20
# optimizer = AdamW(model.parameters(), lr=learning_rate,
#       weight_decay=0.01,
#       betas=(0.9, 0.999),
#       eps=1e-6)

scheduler = get_linear_schedule_with_warmup(optimizer, 200, 11886)

# accum_iter = 2

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()



print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for batch_idx, data in enumerate(tqdm(train_loader)):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        with torch.set_grad_enabled(True):
        
          output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

          # Choose the most probable start position / end position
          start_index = torch.argmax(output.start_logits, dim=1)
          end_index = torch.argmax(output.end_logits, dim=1)
          
          # Prediction is correct only if both start_index and end_index are correct
          train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
          # output.loss = output.loss / 1
          train_loss += output.loss
          
          if fp16_training:
              accelerator.backward(output.loss)
          else:
              output.loss.backward()
          
          # if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
          #   optimizer.step()
          #   optimizer.zero_grad()
          #   scheduler.step()

          optimizer.step()
          optimizer.zero_grad()
          scheduler.step()

          step += 1

          ##### TODO: Apply linear learning rate decay #####

          
          # Print training loss and accuracy over past logging step
          if step % logging_step == 0:
              print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
              train_loss = train_acc = 0

    if validation:
            print("Evaluating Dev Set ...")
            model.eval()
            with torch.no_grad():
                dev_acc = 0
                for i, data in enumerate(tqdm(dev_loader)):
                    output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                            attention_mask=data[2].squeeze(dim=0).to(device))
                    # prediction is correct only if answer text exactly matches
                    res = evaluate(data, output, dev_questions[i]["paragraph_id"], 64, False, n_best)
                    if(res != dev_questions[i]["answer_text"]) :
                      print(f"wrong answering: {i}")
                      print(f"guess: {res}")
                      print(f"answer: {dev_questions[i]['answer_text']}")
                      print()
                    dev_acc += res == dev_questions[i]["answer_text"]
                print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
            model.train()
    

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」


Start Training ...


  0%|          | 0/3962 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 1 | Step 200 | loss = 0.867, acc = 0.691
Epoch 1 | Step 400 | loss = 0.671, acc = 0.738
Epoch 1 | Step 600 | loss = 0.627, acc = 0.741
Epoch 1 | Step 800 | loss = 0.648, acc = 0.759
Epoch 1 | Step 1000 | loss = 0.625, acc = 0.749
Epoch 1 | Step 1200 | loss = 0.588, acc = 0.764
Epoch 1 | Step 1400 | loss = 0.620, acc = 0.752
Epoch 1 | Step 1600 | loss = 0.599, acc = 0.773
Epoch 1 | Step 1800 | loss = 0.585, acc = 0.776
Epoch 1 | Step 2000 | loss = 0.571, acc = 0.777
Epoch 1 | Step 2200 | loss = 0.557, acc = 0.771
Epoch 1 | Step 2400 | loss = 0.588, acc = 0.781
Epoch 1 | Step 2600 | loss = 0.596, acc = 0.765
Epoch 1 | Step 2800 | loss = 0.558, acc = 0.783
Epoch 1 | Step 3000 | loss = 0.541, acc = 0.779
Epoch 1 | Step 3200 | loss = 0.535, acc = 0.787
Epoch 1 | Step 3400 | loss = 0.471, acc = 0.813
Epoch 1 | Step 3600 | loss = 0.471, acc = 0.800
Epoch 1 | Step 3800 | loss = 0.527, acc = 0.794


  0%|          | 0/3962 [00:00<?, ?it/s]

Epoch 2 | Step 200 | loss = 0.269, acc = 0.876
Epoch 2 | Step 400 | loss = 0.235, acc = 0.893
Epoch 2 | Step 600 | loss = 0.295, acc = 0.874
Epoch 2 | Step 800 | loss = 0.290, acc = 0.858
Epoch 2 | Step 1000 | loss = 0.247, acc = 0.889
Epoch 2 | Step 1200 | loss = 0.263, acc = 0.878
Epoch 2 | Step 1400 | loss = 0.248, acc = 0.886
Epoch 2 | Step 1600 | loss = 0.260, acc = 0.881
Epoch 2 | Step 1800 | loss = 0.244, acc = 0.877
Epoch 2 | Step 2000 | loss = 0.243, acc = 0.891
Epoch 2 | Step 2200 | loss = 0.247, acc = 0.891
Epoch 2 | Step 2400 | loss = 0.254, acc = 0.884
Epoch 2 | Step 2600 | loss = 0.243, acc = 0.884
Epoch 2 | Step 2800 | loss = 0.229, acc = 0.894
Epoch 2 | Step 3000 | loss = 0.238, acc = 0.880
Epoch 2 | Step 3200 | loss = 0.239, acc = 0.884
Epoch 2 | Step 3400 | loss = 0.207, acc = 0.907
Epoch 2 | Step 3600 | loss = 0.227, acc = 0.901
Epoch 2 | Step 3800 | loss = 0.263, acc = 0.885


  0%|          | 0/3962 [00:00<?, ?it/s]

Epoch 3 | Step 200 | loss = 0.098, acc = 0.952
Epoch 3 | Step 400 | loss = 0.099, acc = 0.955
Epoch 3 | Step 600 | loss = 0.102, acc = 0.950
Epoch 3 | Step 800 | loss = 0.103, acc = 0.952
Epoch 3 | Step 1000 | loss = 0.096, acc = 0.950
Epoch 3 | Step 1200 | loss = 0.097, acc = 0.949
Epoch 3 | Step 1400 | loss = 0.115, acc = 0.946
Epoch 3 | Step 1600 | loss = 0.095, acc = 0.961
Epoch 3 | Step 1800 | loss = 0.097, acc = 0.953
Epoch 3 | Step 2000 | loss = 0.092, acc = 0.955
Epoch 3 | Step 2200 | loss = 0.080, acc = 0.956
Epoch 3 | Step 2400 | loss = 0.094, acc = 0.957
Epoch 3 | Step 2600 | loss = 0.086, acc = 0.962
Epoch 3 | Step 2800 | loss = 0.087, acc = 0.951
Epoch 3 | Step 3000 | loss = 0.089, acc = 0.959
Epoch 3 | Step 3200 | loss = 0.090, acc = 0.954
Epoch 3 | Step 3400 | loss = 0.079, acc = 0.966
Epoch 3 | Step 3600 | loss = 0.078, acc = 0.964
Epoch 3 | Step 3800 | loss = 0.075, acc = 0.959
Saving Model ...


In [17]:
print("Saving Model ...")
model_save_dir = "./saved_model/luhua_macbert_82775" 
model.save_pretrained(model_save_dir)

Saving Model ...
